In [1]:
# -*- coding: utf-8 -*-
import pymysql
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
curr_dir = '/home/souche/qiongjiu/hgc/'

In [2]:
car_data_columns= ["country_id","manufacturer","battery_quality_mile","battery_quality_time","brand_code","brand_name","car_body",
"cylinder_number","driving_mode","engine_volume_l","fuel_form","gear_box_type","guide_price","height","import_type",
"intake_type","length","level","max_power","max_torque","model_code","model_name","quality_mile","quality_time","rate",
"rate_count","seat_number_top","series_code","series_level","series_name","wheel_base","width","year","down_market_time",
"production_time","shutdown_time","time_to_market","year_1","year_2","year_3","year_4","year_5","year_6","year_7","year_8",
"year_9","year_10","year_11","year_12","year_13","year_14","year_15","year_16"]

model = pd.read_csv(curr_dir+'all_car_model_sep_word_to_sql',sep='\t')


In [6]:
model.loc[model['gear_box_type'] == '待查','gear_box_type'] = '缺失'

In [8]:
# model_code可能是字符串和数字类型混合，统一成字符串类型
model['model_code'] = model['model_code'].astype('str')
# 指导价为0的车型剔除
model = model[model['guide_price']>0]
# 国际档是摩托车，剔除
model = model[model['gear_box_type'] != "国际档"]
model = model.reset_index(drop = True)
### 新车型库 上述情况都没有了哦 ～
model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60741 entries, 0 to 60740
Data columns (total 53 columns):
country_id              60741 non-null object
manufacturer            60741 non-null object
battery_quality_mile    60741 non-null float64
battery_quality_time    60741 non-null float64
brand_code              60741 non-null object
brand_name              60741 non-null object
car_body                60741 non-null object
cylinder_number         60741 non-null float64
driving_mode            60741 non-null object
engine_volume_l         60741 non-null float64
fuel_form               60741 non-null object
gear_box_type           60741 non-null object
guide_price             60741 non-null float64
height                  60741 non-null float64
import_type             60741 non-null object
intake_type             60741 non-null object
length                  60741 non-null float64
level                   60741 non-null object
max_power               60741 non-null float64
max_torqu

In [9]:
# 划分成 “国产” 和 “进口” 两个取值 
model.loc[model['import_type']!='国产', 'import_type'] = '进口'
model['import_type'].value_counts()


国产    41408
进口    19333
Name: import_type, dtype: int64

In [10]:
# # step2: 特征处理，合并一些特征
# def feature_handle(tar):
    
#     if(tar['driving_mode'] == "其他"):
#         tar['driving_mode'] = "缺失"  
#     if (tar['intake_type'] == "其他"):
#         tar['intake_type'] = "缺失"
#     return tar

# model1 = model.apply(feature_handle, axis = 1)
model1 = model.copy()
model1['year'] = model1['year'].astype('int')
model1['guide_price'] = model1['guide_price']/10000
model1['wheel_base'] = model1['wheel_base']/1000
model1['length'] = model1['length']/1000
model1['height'] = model1['height']/1000
model1['width'] = model1['width']/1000
model1['max_torque'] = model1['max_torque']/100
model1['max_power'] = model1['max_power']/100

In [11]:
# 获取车型级别
model1['level'] = list(map(lambda x: x.split('-')[0], model1['series_level']))
# 将 “Mini Bus” 合并到 “A00”
model1.loc[model1['level'] == "Mini Bus", 'level'] = "A00"
# 未使用 'Manufacturer', 'battery_quality_mile', 'battery_quality_time', 'brand_code', 'brand_name', 'car_body',
# 'model_name','rate', 'rate_count',
#'year_1', 'year_2', 'year_3','year_4', 'year_5', 'year_6', 'year_7', 'year_8', 'year_9', 'year_10',
#'year_11', 'year_12', 'year_13', 'year_14', 'year_15', 'year_16'
model_save = model1[['model_code', 'year',  'guide_price', 'wheel_base', 'length', 'height',
                     'width', 'max_torque', 'max_power', 'engine_volume_l', 'cylinder_number',
                     'driving_mode', 'gear_box_type',  'country_id', 'import_type', 'intake_type',
                     'fuel_form', 'series_level', 'level', 'seat_number_top', 'quality_mile',
                     'quality_time', 'year_1', 'year_2', 'year_3', 'year_4', 'year_5', 'year_6',
                     'year_7', 'year_8', 'year_9', 'year_10', 'year_11', 'year_12', 'year_13', 
                     'year_14', 'year_15', 'year_16','car_body','rate','rate_count']]

model_save = model_save.rename(columns={"year": "model_year",
                                        "guide_price": "new_car_price",
                                        "quality_time": "quality_year"})
model_save.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60741 entries, 0 to 60740
Data columns (total 41 columns):
model_code         60741 non-null object
model_year         60741 non-null int64
new_car_price      60741 non-null float64
wheel_base         60741 non-null float64
length             60741 non-null float64
height             60741 non-null float64
width              60741 non-null float64
max_torque         60741 non-null float64
max_power          60741 non-null float64
engine_volume_l    60741 non-null float64
cylinder_number    60741 non-null float64
driving_mode       60741 non-null object
gear_box_type      60741 non-null object
country_id         60741 non-null object
import_type        60741 non-null object
intake_type        60741 non-null object
fuel_form          60741 non-null object
series_level       60741 non-null object
level              60741 non-null object
seat_number_top    60741 non-null float64
quality_mile       60741 non-null float64
quality_year       6

In [12]:
data_filter = {
'driving_mode':['中置前驱','中置后驱','中置四驱','前置前驱','前置后驱','前置四驱','双电机四驱','后置后驱','后置四驱'],
'gear_box_type' : ['E-CVT+自动变速箱','ISR变速箱','双离合变速箱(DCT)','固定齿比变速箱','序列变速箱(AMT)','手动变速箱(MT)','无级变速箱(CVT)','机械式自动变速箱(AMT)','电子无级变速箱(E-CVT)','自动变速箱(AT)'],
'country_id' : ['AMERICA','CHINA','CZECH','ENGLAND','FRENCH','GERMANY','ITALIA','JAPAN','KOREA','NETHERLANDS','ROMANIA','ROSSIA','SPAIN','SWEDEN'],
'import_type' : ['国产', '进口'],
'intake_type' : ['双涡轮增压', '四涡轮增压','机械+涡轮增压', '机械增压', '涡轮增压', '自然吸气','无'],
'fuel_form' : ['增程式', '插电式混合动力', '柴油', '汽油', '汽油+48V轻混系统', '油气混合', '油电混合', '纯电动'],
'level' : ['A', 'A0', 'A00', 'B', 'BUS', 'C', 'D', 'E'],
'car_body' :['MPV','SUV','三厢车','两厢车','客车','掀背车','旅行车','皮卡','硬顶敞篷车','硬顶跑车','软顶敞篷车'],
'series_level' : ['B-M SUV', 'A0-H SUV', 'A-H MPV', 'D-Top', 'D-H', 'C-Lux SUV', 'A00-M', 'A-H SUV', 'B-Lux', 'B-H', 'A-L', 'C-H SUV', 'B-M MPV', 'A0-H MPV', 'C-H', 'A0-L SUV', 'A0-L', 'B-H MPV', 'A-M MPV', 'B-L SUV', 'A00-H SUV', 'C-Lux MPV', 'A-Lux', 'A00-H', 'A-L MPV', 'A-M', 'A0-M', 'A0-Lux SUV', 'B-Top', 'D-Lux', 'A00-L SUV', 'C-H MPV', 'D-H SUV', 'A-L SUV ', 'A0-L MPV', 'B-L MPV', 'C-Top', 'A0-Lux', 'Mini Bus', 'D-Lux MPV', 'D-Top SUV', 'B-L', 'A-H', 'A00-L MPV', 'A0-Lux MPV', 'A-L SUV', 'E-Top', 'A00-Lux', 'B-M', 'B-H SUV', 'A00-L', 'BUS', 'B-Lux MPV', 'D-Lux SUV', 'A-M SUV', 'B-Lux SUV', 'A0-M SUV', 'A-Lux MPV', 'A0-H', 'A0-M MPV', 'C-Lux', 'A-Lux SUV']
}

for col,col_value in data_filter.items():
    model_save.loc[~model_save[col].isin(col_value),col] = '缺失'

In [13]:
# 该函数的作用是 将变量进行onehot 处理
from sklearn import preprocessing
def feature_onehot(traindata, feature_list):
    for feature in feature_list:
        le = preprocessing.LabelEncoder()
        le.fit(traindata[[feature]])
        traindata[feature+"_tem"] = le.transform(traindata[[feature]])
        enc = preprocessing.OneHotEncoder()
        enc.fit(traindata[[feature+"_tem"]])
        pro_onehot = enc.transform(traindata[[feature+"_tem"]]).toarray()
        for i in range(0,pro_onehot.shape[1]):
            traindata[feature+str(i)] = pro_onehot[:,i]
        traindata = traindata.drop([feature+"_tem"],axis=1)
    return traindata
feature_list = ["driving_mode", "gear_box_type", "country_id", 
                "import_type", "intake_type", "fuel_form", "level",'car_body','series_level']
model_save = feature_onehot(model_save, feature_list)

In [14]:
# 保存版本车型数据
model_save.to_csv(curr_dir+"2021-02-02版车型参数及独热编码.csv", header=True, index=False)